# ¿Cómo podemos tomar la mejor decisión con información incompleta?

Esta es justamente la pregunta que trata de resolver la teoría de juegos de información incompleta. Tomar la mejor decisión en juego que implica un conflicto de intereses, involucra no ser predecibles, ya que de lo contrario nos convertiríamos en explotables. Otro agente del entrono podría preever nuestras acciones de antemano y modificar su estrategia en conscuencia para maximizar su beneficio; más si el juego es de suma 0 que aparte de maximizar su beneficio, intentará minimizar el nuestro.


Un algoritmo simple que propone el cálculo de estrategias óptimas en juegos de suma cero, es el regret matching. Mediante simulaciones, pretende actualizar las estrategias de los agentes, con el objetivo de converger a un equilibrio de Nash. La lógica es la siguiente: ir midiendo lo que habríamos ganado de haber elegido una acción distinta en el pasado, si todo se hubiera mantenido igual. 

## Ejemplo Trabajado: Khun Poker

In [1]:
import numpy as np 
import random

In [2]:
# Kuhn Poker Setup
cards_values = { "J": 1, "Q": 2, "K": 3 }
HANDS        = [ 'JQ', 'JK', 'QJ', 'KJ', 'QK', 'KQ' ]
ACTIONS      = { 0: 'pass', 1: 'bet' }
STATES       = [ c + ' ' + s for c in cards_values.keys() for s in ['', '0', '1','01'] ]
TERMINALS    = [ h + ' ' + s for h in HANDS for s in ['11','00','010', '10', '011'] ]

In [ ]:
# Funciones del juego
def deal():
    """Selecciona una mano aleatoria de las posibles manos."""
    return random.choice(HANDS)

def payments(hands, h):
    """Calcula el pago basado en las manos y el historial de acciones."""
    if h not in TERMINALS:
        return 0
    payment = 2 if h.count('1') < 2 else 4
    return -payment if (h[-2:] == '01' or (cards_values[hands[1]] > cards_values[hands[0]] and h[-2] == h[-1])) else payment

In [3]:
# Inicialización de arrepentimientos y política
regrets      = {state: np.zeros(len(ACTIONS)) for state in STATES}
strategies   = {state: np.zeros(len(ACTIONS)) for state in STATES}

def get_strategy(state):
    """Calcula la estrategia para un estado dado basado en los arrepentimientos acumulados."""
    state_regrets = regrets[state]
    s = sum([ 0 if state_regrets[i] <=0 else state_regrets[i] for i in range(len(state_regrets))])
    if s  > 0:
        strategy = [ r/s if r > 0 else 0 for r in state_regrets] 
    else:
        strategy = np.ones(len(ACTIONS)) / len(ACTIONS)
    strategies[state] += strategy  # Acumula la estrategia
    return strategy

def get_action(strategy):
    """Elige una acción basada en la distribución de probabilidad de la estrategia."""
    return np.random.choice(list(ACTIONS.keys()), p=strategy)

In [4]:
# Entrenamiento con el algoritmo
iters = 100_000
for t in range(iters):
    cards   = deal()
    history = cards + ' '
    actions_ = []

    for player in range(2):
        
        if history in TERMINALS:
            break

        state    = cards[player] + ' ' + history[3:]
        strategy = get_strategy(state)
        action   = get_action(strategy)
        actions_.append(action)
        history += str(action)

    # Ejecutar acción pendiente si no es terminal
    if not history in TERMINALS:
        state    = cards[0] + ' ' + history[3:]
        strategy = get_strategy(state)
        action   = get_action(strategy)
        actions_.append(action)
        history += str(action)

    payment = payments(cards, history)

    # Actualización de arrepentimientos
    for i in range(len(actions_)):
        idx = i % 2
        state = cards[idx] + ' ' + ''.join(map(str, actions_[:i]))
        for a in ACTIONS.keys():
            hypothetical_history = cards + ' ' + ''.join(map(str, actions_[:i] + [a] + actions_[i+1:]))
            # recodificación del historial en caso especial
            hypothetical_history = hypothetical_history.replace('110', '11')                                    
            hypothetical_payment = payments(cards, hypothetical_history)
            regret = hypothetical_payment - payment if idx == 0 else -hypothetical_payment + payment
            regrets[state][a] += regret

In [5]:
strategies

{'J ': array([16858.86942927, 16362.13057073]),
 'J 0': array([10476.33339138,  7414.66660862]),
 'J 1': array([1.56555e+04, 1.50000e+00]),
 'J 01': array([170.,   1.]),
 'Q ': array([18892.39636803, 14462.60363197]),
 'Q 0': array([16397.3559273,   337.6440727]),
 'Q 1': array([6.14285714e+00, 1.65218571e+04]),
 'Q 01': array([   4., 3901.]),
 'K ': array([16712., 16712.]),
 'K 0': array([1.7677e+04, 1.0000e+00]),
 'K 1': array([5.00000e-01, 1.55105e+04]),
 'K 01': array([2.000e+00, 3.625e+03])}

In [6]:
def avg_strategy(strategy_sum):
    """Calcula la estrategia promedio a partir de la suma de estrategias."""
    total = sum(strategy_sum)
    return [s / total for s in strategy_sum] if total > 0 else [1 / len(strategy_sum) for _ in strategy_sum]

average_strategies = {k: avg_strategy(v) for k, v in strategies.items()}
average_strategies

{'J ': [0.5074762779346557, 0.49252372206534417],
 'J 0': [0.5855644397395475, 0.41443556026045253],
 'J 1': [0.9999041962061698, 9.580379383023568e-05],
 'J 01': [0.9941520467836257, 0.005847953216374269],
 'Q ': [0.5664037286173996, 0.4335962713826004],
 'Q 0': [0.9798240769227249, 0.020175923077275157],
 'Q 1': [0.00037166367030839434, 0.9996283363296915],
 'Q 01': [0.001024327784891165, 0.9989756722151089],
 'K ': [0.5, 0.5],
 'K 0': [0.9999434325149904, 5.656748500961647e-05],
 'K 1': [3.223518793114564e-05, 0.9999677648120688],
 'K 01': [0.0005514199062586159, 0.9994485800937414]}